In [4]:
%matplotlib inline

import os
import csv
import glob
import numpy as np
import matplotlib.pyplot as plt
import heartpy as hp
import pandas as pd
pd.set_option("display.precision", 2)
from scipy.interpolate import interp1d
from utils.process_ppg import load_PPG_signal, get_filtered_ppg, get_ppg_measures_batch

%matplotlib inline

In [6]:
def calc_HRV_measures(signal,sample_rate=64.0):
    t_elapsed = np.arange(0,len(signal))
    t_elapsed = t_elapsed/sample_rate
    #filtered = get_filtered_ppg(signal, t_elapsed, sample_rate=sample_rate)
    filtered = get_filtered_ppg(signal, sample_rate=sample_rate)
    if len(filtered) > 30*sample_rate:
        start_indx = int(((len(filtered)//2) - (15*sample_rate)))
        end_indx = int(((len(filtered)//2) + (15*sample_rate)))
        filtered = filtered[start_indx : end_indx]
    plt.figure(figsize=(12,4))
    x_axis = [i/sample_rate for i in range(len(filtered))]
    #plt.plot(x_axis, filtered)
    #plt.xlabel('Time(seconds)')
    #plt.title('Filtered PPG Signal: ')

    wd, m = hp.process(filtered, sample_rate = sample_rate)

   # for measure in m.keys():
   #     print(measure, m[measure])

    return m

def calc_diff(baseline_sig,game_sig):
    base_dict = calc_HRV_measures(baseline_sig)
    game_dict = calc_HRV_measures(game_sig)
    perc_diff = {}
    abs_diff = {}
    for key in base_dict:
        perc_diff[key] = game_dict[key]/base_dict[key]
        abs_diff[key] = game_dict[key] - base_dict[key]

    #print(perc_diff)
    #print(abs_diff)
    return perc_diff,abs_diff



folder = './Data'
HR_path = folder+"/**/HR.csv"
PPG_path = folder + "/**/BVP.csv"
tag_path = folder + "/**/tags.csv"

#Load raw PPG data
PPG_csv_paths = glob.glob(PPG_path, recursive=True)

#Load timestamp
tag_csv_paths = glob.glob(tag_path,recursive=True)

id_list = []
df_list = []
tag_list = []
sample_rate = 0

for csv_path in PPG_csv_paths:
    curr_df = pd.read_csv(csv_path)
    id_num = curr_df.columns.values.tolist()
    id_list.append(id_num)
    sample_rate = curr_df.iloc[0]
    df_list.append(curr_df)

for i,tag_path in enumerate(tag_csv_paths):
    curr_tag_df = pd.read_csv(tag_path,names=id_list[i])
    tag_list.append(curr_tag_df)

merged_tag_df = pd.concat(tag_list,axis=1)
merged_df = pd.concat(df_list,axis=1)
diff_tag_df = merged_tag_df - merged_tag_df.iloc[0]
#ppg_df = merged_df.iloc[1: , :]

save_dict = {}
HR_df = pd.DataFrame()

for i,csv_path in enumerate(PPG_csv_paths):
    print(i)
    bvp_df = pd.read_csv(csv_path)
    sample_rate = 64.0 
    bvp_df = bvp_df.iloc[1:]
    bvp_df
    
    id = id_list[i][0]
    raw_signal = bvp_df.to_numpy()
    t_elapsed = np.arange(0,len(raw_signal))
    t_elapsed = t_elapsed/64
    raw_signal = np.squeeze(raw_signal)
    #Segment 
    times = diff_tag_df[id]
    baseline_sig = raw_signal[t_elapsed<times[1]]
    game_sig = raw_signal[(times[1]<t_elapsed) * (t_elapsed<times[2])]
    post_game_sig = raw_signal[(times[2]<t_elapsed)]
    try:
        perc,abs = calc_diff(baseline_sig,game_sig)
        for key in perc:
            perc_col_name = 'perc_' + key
            abs_col_name = 'abs_' + key
            HR_df[id][perc_col_name] = 
        
        diff_dict = {'perc':perc,'abs':abs}
        save_dict[str(id)] = diff_dict

        print(perc)
        print(abs)
    except:
        print("An exception occurred")

0
{'bpm': 0.9950476190476192, 'ibi': 1.0049770290964777, 'sdnn': 0.43406973434965723, 'sdsd': 0.5704596737172418, 'rmssd': 0.6074153845190294, 'pnn20': 0.892156862745098, 'pnn50': 0.5634674922600619, 'hr_mad': 1.0, 'sd1': 0.6090306885879413, 'sd2': 0.3410449266596469, 's': 0.20770682652294867, 'sd1/sd2': 1.7857784736840157, 'breathingrate': 1.1192375237816479}
{'bpm': -0.39315685845546966, 'ibi': 3.7615740740740193, 'sdnn': -59.25233280684644, 'sdsd': -26.189162302888015, 'rmssd': -42.13945456193218, 'pnn20': -0.09954751131221728, 'pnn50': -0.3190045248868778, 'hr_mad': 0.0, 'sd1': -29.59578852758935, 'sd2': -83.14489556231314, 's': -23774.030684350437, 'sd1/sd2': 0.4714195480020613, 'breathingrate': 0.023386785090055484}
1
{'bpm': 1.0393984962406015, 'ibi': 0.9620949074074074, 'sdnn': 0.510018021058691, 'sdsd': 0.4232073695151589, 'rmssd': 0.6132441406718666, 'pnn20': 1.35, 'pnn50': 1.2, 'hr_mad': 1.0, 'sd1': 0.6248199740725324, 'sd2': 0.43603010147093424, 's': 0.27244031669591284, 's

C:\Users\Rob\anaconda3\envs\HRI\lib\site-packages\scipy\interpolate\_fitpack2.py:280: UserWarning: 
The maximal number of iterations maxit (set to 20 by the program)
allowed for finding a smoothing spline with fp=s has been reached: s
too small.
There is an approximation returned but the corresponding weighted sum
of squared residuals does not satisfy the condition abs(fp-s)/s < tol.
  warnings.warn(message)
C:\Users\Rob\anaconda3\envs\HRI\lib\site-packages\scipy\interpolate\_fitpack2.py:280: UserWarning: 
The maximal number of iterations maxit (set to 20 by the program)
allowed for finding a smoothing spline with fp=s has been reached: s
too small.
There is an approximation returned but the corresponding weighted sum
of squared residuals does not satisfy the condition abs(fp-s)/s < tol.
  warnings.warn(message)
C:\Users\Rob\anaconda3\envs\HRI\lib\site-packages\scipy\interpolate\_fitpack2.py:280: UserWarning: 
The maximal number of iterations maxit (set to 20 by the program)
allowed fo

{'bpm': 1.0421270718232045, 'ibi': 0.9595758780649437, 'sdnn': 0.8824727995964522, 'sdsd': 0.9929083705024573, 'rmssd': 1.0046917444325731, 'pnn20': 1.0875000000000001, 'pnn50': 1.0285714285714287, 'hr_mad': 1.25, 'sd1': 1.0340677301816685, 'sd2': 0.6575050183550015, 's': 0.6799047219134128, 'sd1/sd2': 1.5727145821164705, 'breathingrate': 1.3896505970809376}
{'bpm': 3.537669013543052, 'ibi': -28.88257575757575, 'sdnn': -15.895477887491793, 'sdsd': -0.6650682236613648, 'rmssd': 0.8203422444970272, 'pnn20': 0.07777777777777783, 'pnn50': 0.022222222222222254, 'hr_mad': 15.625, 'sd1': 4.080172908771104, 'sd2': -53.365981509166616, 's': -18766.13336017565, 'sd1/sd2': 0.4402134636300413, 'breathingrate': 0.09921084585128903}
7
{'bpm': 1.817241379310345, 'ibi': 0.5502846299810247, 'sdnn': 1.5399844089221062, 'sdsd': 3.7171124877822193, 'rmssd': 4.349980470885746, 'pnn20': 1.5333333333333334, 'pnn50': 2.625, 'hr_mad': 1.8, 'sd1': 4.364462755479857, 'sd2': 0.9690686699715793, 's': 4.22946411759

C:\Users\Rob\anaconda3\envs\HRI\lib\site-packages\scipy\interpolate\_fitpack2.py:280: UserWarning: 
The maximal number of iterations maxit (set to 20 by the program)
allowed for finding a smoothing spline with fp=s has been reached: s
too small.
There is an approximation returned but the corresponding weighted sum
of squared residuals does not satisfy the condition abs(fp-s)/s < tol.
  warnings.warn(message)
C:\Users\Rob\AppData\Local\Temp\ipykernel_2800\2238449296.py:10: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure(figsize=(12,4))
C:\Users\Rob\anaconda3\envs\HRI\lib\site-packages\scipy\interpolate\_fitpack2.py:280: UserWarning: 
The maximal number of iterations maxit (set to 20 by the program)
allowed for finding a smoothing spline with fp=s has been r

{'bpm': 1.0886402753872633, 'ibi': 0.9185770750988143, 'sdnn': 1.8942654357754942, 'sdsd': 1.2356449652607502, 'rmssd': 0.9969720630047815, 'pnn20': 0.7, 'pnn50': 0.5454545454545455, 'hr_mad': 1.3333333333333333, 'sd1': 0.9959927954185247, 'sd2': 2.2397474811360536, 's': 2.230772354768298, 'sd1/sd2': 0.444689771417148, 'breathingrate': 0.5206370198930952}
{'bpm': 6.188702863401659, 'ibi': -69.9728260869565, 'sdnn': 39.7452407446783, 'sdsd': 7.147833147199599, 'rmssd': -0.15243526835364918, 'pnn20': -0.2142857142857143, 'pnn50': -0.1515151515151515, 'hr_mad': 7.8125, 'sd1': -0.1420858447343818, 'sd2': 59.10041719214168, 's': 6535.7317035431115, 'sd1/sd2': -0.4130357653449422, 'breathingrate': -0.10147395853236765}
13
{'bpm': 0.6049049820236261, 'ibi': 1.6531521969857779, 'sdnn': 0.8716409102057451, 'sdsd': 0.7370576424228761, 'rmssd': 0.7102835986151549, 'pnn20': 0.9810606060606061, 'pnn50': 0.9249999999999999, 'hr_mad': 0.6666666666666666, 'sd1': 0.6528485517685442, 'sd2': 0.6550158115

...

<Figure size 864x288 with 0 Axes>

<Figure size 864x288 with 0 Axes>

<Figure size 864x288 with 0 Axes>

<Figure size 864x288 with 0 Axes>

<Figure size 864x288 with 0 Axes>

In [13]:
HR_df = pd.DataFrame.from_dict(save_dict,orient='index')
print(HR_df)
print(HR_df.columns)

                                                            perc  \
1647264808.00  {'bpm': 0.9950476190476192, 'ibi': 1.004977029...   
1647265213.00  {'bpm': 1.0393984962406015, 'ibi': 0.962094907...   
1647265691.00  {'bpm': 0.5545822102425876, 'ibi': 1.803159173...   
1647267334.00  {'bpm': 0.8703296703296705, 'ibi': 1.148989898...   
1647267651.00  {'bpm': 0.9380549909061732, 'ibi': 1.066035583...   
1647268069.00  {'bpm': 0.9025270758122743, 'ibi': 1.108, 'sdn...   
1647268497.00  {'bpm': 1.0421270718232045, 'ibi': 0.959575878...   
1647268988.00  {'bpm': 1.817241379310345, 'ibi': 0.5502846299...   
1647269598.00  {'bpm': 1.116485403548941, 'ibi': 0.8956677774...   
1646752964.00  {'bpm': 0.7887429643527204, 'ibi': 1.267840152...   
1646754723.00  {'bpm': 1.053153153153153, 'ibi': 0.9495295124...   
1646755454.00  {'bpm': 1.0042541320873142, 'ibi': 0.995763888...   
1646756026.00  {'bpm': 1.0886402753872633, 'ibi': 0.918577075...   
1646756620.00  {'bpm': 0.6049049820236261, 'ibi'

In [18]:
print(save_dict)
print(save_dict.keys())
print(save_dict.values())

{'1647264808.00': {'perc': {'bpm': 0.9950476190476192, 'ibi': 1.0049770290964777, 'sdnn': 0.43406973434965723, 'sdsd': 0.5704596737172418, 'rmssd': 0.6074153845190294, 'pnn20': 0.892156862745098, 'pnn50': 0.5634674922600619, 'hr_mad': 1.0, 'sd1': 0.6090306885879413, 'sd2': 0.3410449266596469, 's': 0.20770682652294867, 'sd1/sd2': 1.7857784736840157, 'breathingrate': 1.1192375237816479}, 'abs': {'bpm': -0.39315685845546966, 'ibi': 3.7615740740740193, 'sdnn': -59.25233280684644, 'sdsd': -26.189162302888015, 'rmssd': -42.13945456193218, 'pnn20': -0.09954751131221728, 'pnn50': -0.3190045248868778, 'hr_mad': 0.0, 'sd1': -29.59578852758935, 'sd2': -83.14489556231314, 's': -23774.030684350437, 'sd1/sd2': 0.4714195480020613, 'breathingrate': 0.023386785090055484}}, '1647265213.00': {'perc': {'bpm': 1.0393984962406015, 'ibi': 0.9620949074074074, 'sdnn': 0.510018021058691, 'sdsd': 0.4232073695151589, 'rmssd': 0.6132441406718666, 'pnn20': 1.35, 'pnn50': 1.2, 'hr_mad': 1.0, 'sd1': 0.624819974072532

In [25]:
print(HR_df.k

<bound method NDFrame.keys of                                                             perc  \
1647264808.00  {'bpm': 0.9950476190476192, 'ibi': 1.004977029...   
1647265213.00  {'bpm': 1.0393984962406015, 'ibi': 0.962094907...   
1647265691.00  {'bpm': 0.5545822102425876, 'ibi': 1.803159173...   
1647267334.00  {'bpm': 0.8703296703296705, 'ibi': 1.148989898...   
1647267651.00  {'bpm': 0.9380549909061732, 'ibi': 1.066035583...   
1647268069.00  {'bpm': 0.9025270758122743, 'ibi': 1.108, 'sdn...   
1647268497.00  {'bpm': 1.0421270718232045, 'ibi': 0.959575878...   
1647268988.00  {'bpm': 1.817241379310345, 'ibi': 0.5502846299...   
1647269598.00  {'bpm': 1.116485403548941, 'ibi': 0.8956677774...   
1646752964.00  {'bpm': 0.7887429643527204, 'ibi': 1.267840152...   
1646754723.00  {'bpm': 1.053153153153153, 'ibi': 0.9495295124...   
1646755454.00  {'bpm': 1.0042541320873142, 'ibi': 0.995763888...   
1646756026.00  {'bpm': 1.0886402753872633, 'ibi': 0.918577075...   
1646756620.00  {'b